## Startup

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.stats import norm
import scipy.stats as scs


import os
import pickle

In [2]:
np.random.seed(42)

In [3]:
from scripts.params import get_params

params = get_params()

In [18]:
dataroute = os.path.join("..", "data")
processedroute = os.path.join("...", "processed")
resultsroute = os.path.join("..", "results")
descriptivegraphsroute=os.path.join(resultsroute, "graphs", "descriptive")

## Data Retrieval

In [5]:
name = f'finaldf_train_{params["tablename"]}.pickle'
filename = os.path.join(dataroute, name)
with open(filename, "rb") as handle:
    df = pickle.load(handle)

In [6]:
df.head()

,^MERV_rets,^MERV_log_rets,^MERV_gk_vol,GGAL.BA_rets,GGAL.BA_log_rets,GGAL.BA_gk_vol,GGAL_rets,GGAL_log_rets,GGAL_gk_vol,YPFD.BA_rets,...,BBAR.BA_gk_vol,BBAR_rets,BBAR_log_rets,BBAR_gk_vol,USD_rets,USD_log_rets,USD_gk_vol,USD_^MERV_rets,USD_^MERV_log_rets,USD_^MERV_gk_vol
2013-01-03,0.007552,0.007524,0.000129,0.010616,0.010560,0.000677,-0.012748,-0.012830,0.001228,-0.006862,...,0.000169,-0.005725,-0.005742,0.000960,0.008830,0.008792,0.000014,0.001247,0.001246,0.000129
2013-01-04,0.007092,0.007067,0.000158,-0.006303,-0.006323,0.000208,-0.010043,-0.010094,0.000554,0.004936,...,0.000406,-0.019194,-0.019381,0.000635,0.018043,0.017883,0.000133,-0.005727,-0.005744,0.000158
2013-01-07,-0.001035,-0.001035,0.000022,0.002114,0.002112,0.000063,-0.014493,-0.014599,0.000517,0.010805,...,0.000492,0.015656,0.015534,0.000511,-0.002489,-0.002492,0.000048,-0.009769,-0.009817,0.000022
2013-01-08,0.008285,0.008251,0.000082,-0.008439,-0.008475,0.000153,-0.016176,-0.016309,0.001085,0.049563,...,0.000438,-0.015414,-0.015534,0.000642,0.015356,0.015239,0.000064,-0.001117,-0.001118,0.000082
2013-01-09,0.017826,0.017669,0.000273,0.000000,0.000000,0.000000,0.011958,0.011887,0.005238,0.000000,...,0.000000,-0.003914,-0.003921,0.000147,-0.008671,-0.008709,0.001065,0.017245,0.017098,0.000273


## Descriptive graphs

In [24]:
log_rets_list=[]
vol_list=[]
for column in df.columns: 
    if column.endswith("log_rets"):
        log_rets_list.append(column)
    if column.endswith("gk_vol"):
        vol_list.append(column)

In [25]:
sb.set_style(style='darkgrid')
sb.set_palette(sb.color_palette(palette='deep'))

In [26]:
for column in log_rets_list:
    fig=df[column].plot(title=column).get_figure()
    fig.savefig(os.path.join(descriptivegraphsroute, "log_rets", f"{column}.png"))
    plt.close()

In [27]:
for column in vol_list:
    fig=df[column].plot(title=column).get_figure()
    fig.savefig(os.path.join(descriptivegraphsroute, "gk_vol", f"{column}.png"))
    plt.close()

### Autocorrelograms

In [38]:
acf_lags=252

In [39]:
def save_acf(column, path):
    fig = plt.figure()
    ax1 = fig.add_subplot(1, 1, 1)
    plot_acf(df[column],
                lags=acf_lags,                 # Cuantos lags busco autocorrelacionar
                zero=False,                # Si tomo el lag cero
                alpha=0.05,                # El rango de certeza marcado en azul
                use_vlines=False,          # Lineas verticales que conectan cada punto con el eje x
                ax=ax1)                    # La posicion en la figura
    ax1.grid(True)
    ax1.set_xlabel('Lag')
    ax1.set_ylabel('Autocorrelation')
    ax1.set_title(f'Autocorrelation of {column}')
    fig.savefig(path)
    plt.close()


In [40]:
for column in log_rets_list:
    save_acf(column, 
             os.path.join(
                 descriptivegraphsroute, 
                 "acf", "log_rets", f"{column}.png"))

In [41]:
for column in vol_list:
    save_acf(column, 
             os.path.join(
                 descriptivegraphsroute, 
                 "acf", "gk_vol", f"{column}.png"))

In [45]:
def save_pacf(column, path):
    """
    """
    fig = plt.figure(figsize=[13.6, 10.2])
    ax1 = fig.add_subplot(1, 1, 1)
    plot_pacf(df[column],
                method='ywa',                 # Metodo de Yule Walker con correccion de sesgo por autocovarianzas
                lags=acf_lags,                # Cuantos lags busco autocorrelacionar
                zero=False,                   # Si tomo el lag cero
                alpha=0.05,                   # El rango de certeza marcado en azul
                use_vlines=False,             # Lineas verticales que conectan cada punto con el eje x
                ax=ax1)                       # La posicion en la figura
    ax1.grid(True)
    ax1.set_xlabel('Lag')
    ax1.set_ylabel('Partial Autocorrelation')
    ax1.set_title(f'Partial Autocorrelation of {column}')

    fig.savefig(path)
    plt.close()

In [46]:
for column in log_rets_list:
    save_pacf(column, 
             os.path.join(
                 descriptivegraphsroute, 
                 "pacf", "log_rets", f"{column}.png"))

In [48]:
for column in vol_list:
    save_acf(column, 
             os.path.join(
                 descriptivegraphsroute, 
                 "pacf", "gk_vol", f"{column}.png"))

In [66]:
def save_hist_normal(column, path):
    fig = plt.figure(figsize=[13.6, 5.1])
    ax1 = fig.add_subplot(1, 1, 1)

    sb.distplot(df[column].fillna(0),
                ax=ax1,
                hist=True,
                bins=int(np.ceil(np.log2(len(df[column].index)) + 15)),
                label='Observed data KDE',
                fit=norm,
                fit_kws = {"color":"r", "lw":3, "label":"Fitted Normal"})
    plt.legend()
    plt.grid(True)
    plt.xlabel('Log Returns')
    plt.ylabel('Frequency')
    plt.title(f'Histogram for return frequency for {column}')
    fig.savefig(path)
    plt.close()

In [67]:
for column in log_rets_list:
    save_hist_normal(
        column,
        os.path.join(
            descriptivegraphsroute, 
            "histograms", f"{column}.png"))

C:\Users\Alfred\AppData\Local\Temp\ipykernel_1900\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),


C:\Users\Alfred\AppData\Local\Temp\ipykernel_1900\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),
C:\Users\Alfred\AppData\Local\Temp\ipykernel_1900\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),
C:\Users\Alfred\AppData\Loca

In [68]:
for column in vol_list:
    save_hist_normal(
        column,
        os.path.join(
            descriptivegraphsroute, 
            "histograms", f"{column}.png"))

C:\Users\Alfred\AppData\Local\Temp\ipykernel_1900\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),
C:\Users\Alfred\AppData\Local\Temp\ipykernel_1900\2059545607.py:5: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sb.distplot(df[column].fillna(0),
C:\Users\Alfred\AppData\Loca

In [73]:
def analyze_skew_kurt(dataframe):
    results = pd.DataFrame(index=dataframe.columns, 
                           columns=['Skewness', 'Skewness P-Value', 'Kurtosis', 'Kurtosis P-Value', 'Normaltest Stat', 'Normaltest P-Value'])
    for column in dataframe.columns:
        skew_val, skew_pval = scs.skewtest(dataframe[column], nan_policy='omit')
        kurt_val, kurt_pval = scs.kurtosistest(dataframe[column], nan_policy='omit')
        normtest_stat, normtest_pval = scs.normaltest(dataframe[column], nan_policy='omit')
        results.loc[column] = [skew_val, skew_pval, kurt_val, kurt_pval, normtest_stat, normtest_pval]

    return results

In [74]:
analyze_skew_kurt(df[log_rets_list].fillna(0))

,Skewness,Skewness P-Value,Kurtosis,Kurtosis P-Value,Normaltest Stat,Normaltest P-Value
^MERV_log_rets,-33.567857,0.0,29.397219,0.0,1990.997461,0.0
GGAL.BA_log_rets,-31.284098,0.0,29.15781,0.0,1828.872682,0.0
GGAL_log_rets,-39.96483,0.0,31.664765,0.0,2599.845002,0.0
YPFD.BA_log_rets,-10.451265,0.0,19.293129,0.0,481.45375,0.0
YPF_log_rets,-21.90325,0.0,24.660962,0.0,1087.915436,0.0
EDN.BA_log_rets,-27.398942,0.0,28.678056,0.0,1573.132904,0.0
EDN_log_rets,-36.204989,0.0,31.096108,0.0,2277.769119,0.0
BMA.BA_log_rets,-27.939471,0.0,28.118831,0.0,1571.282667,0.0
BMA_log_rets,-34.88319,0.0,30.428483,0.0,2142.729527,0.0
BBAR.BA_log_rets,6.005963,0.0,16.785525,0.0,317.825448,0.0
